In [1]:
#!pip install --upgrade tensorflow

import numpy as np
import pandas as pd
import numpy as np
import tensorflow
import keras
import string 
import requests

#pip install np_utils
#rom tensorflow.keras.utils import plot_model

#plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [2]:
from numpy import array
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.python.keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

In [3]:
data=""""Orientation - Terms And Concepts IN System Development 
The purpose of the sub-part is to orientate oneself generally among different different terms and concepts in system development and does not need to be overworked in any way, everyone who has participated and collaborated with their group will be approved in this part. Those who can briefly and in their own words explain a complicated subject in a simple and understandable language are VG candidates. However, it is the ability to analyze and object model the task that will be decisive for the final grade in Introduction to programming.

Team Database: This group delves into databases, each member chooses a database to delve into and writes a summary of the database's most important features, e.g. how to download installs and configures, it is possible to.comply within the group with the same database. The group compiles a document and reports the results."""

In [4]:
data=data.lower()
data

'"orientation - terms and concepts in system development \nthe purpose of the sub-part is to orientate oneself generally among different different terms and concepts in system development and does not need to be overworked in any way, everyone who has participated and collaborated with their group will be approved in this part. those who can briefly and in their own words explain a complicated subject in a simple and understandable language are vg candidates. however, it is the ability to analyze and object model the task that will be decisive for the final grade in introduction to programming.\n\nteam database: this group delves into databases, each member chooses a database to delve into and writes a summary of the database\'s most important features, e.g. how to download installs and configures, it is possible to.comply within the group with the same database. the group compiles a document and reports the results.'

In [5]:
def del_punctuation(data):
    return data.translate(str.maketrans("","", string.punctuation))

In [6]:

# integer encode text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
encoded = tokenizer.texts_to_sequences([data])[0]
encoded

[24,
 10,
 1,
 11,
 3,
 12,
 13,
 2,
 25,
 14,
 2,
 26,
 15,
 7,
 4,
 27,
 28,
 29,
 30,
 16,
 16,
 10,
 1,
 11,
 3,
 12,
 13,
 1,
 31,
 32,
 33,
 4,
 8,
 34,
 3,
 35,
 36,
 37,
 17,
 38,
 39,
 1,
 40,
 18,
 19,
 6,
 20,
 8,
 41,
 3,
 21,
 15,
 42,
 17,
 43,
 44,
 1,
 3,
 19,
 45,
 46,
 47,
 5,
 48,
 49,
 3,
 5,
 50,
 1,
 51,
 52,
 53,
 54,
 55,
 56,
 22,
 7,
 2,
 57,
 4,
 58,
 1,
 59,
 60,
 2,
 61,
 62,
 20,
 8,
 63,
 64,
 2,
 65,
 66,
 3,
 67,
 4,
 68,
 69,
 9,
 21,
 6,
 70,
 23,
 71,
 72,
 73,
 74,
 5,
 9,
 4,
 75,
 23,
 1,
 76,
 5,
 77,
 14,
 2,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 4,
 85,
 86,
 1,
 87,
 22,
 7,
 88,
 4,
 89,
 90,
 2,
 6,
 18,
 2,
 91,
 9,
 2,
 6,
 92,
 5,
 93,
 1,
 94,
 2,
 95]

In [7]:
allVocab_size=len(encoded)
print('AllVocabulary Size: %d' % allVocab_size)


AllVocabulary Size: 152


In [8]:
# determine the vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)



Vocabulary Size: 96


In [9]:
# create word -> word sequences
sequences = list()
for i in range(1, len(encoded)):
	sequence = encoded[i-1:i+1]
	sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 151


In [10]:
sequences

[[24, 10],
 [10, 1],
 [1, 11],
 [11, 3],
 [3, 12],
 [12, 13],
 [13, 2],
 [2, 25],
 [25, 14],
 [14, 2],
 [2, 26],
 [26, 15],
 [15, 7],
 [7, 4],
 [4, 27],
 [27, 28],
 [28, 29],
 [29, 30],
 [30, 16],
 [16, 16],
 [16, 10],
 [10, 1],
 [1, 11],
 [11, 3],
 [3, 12],
 [12, 13],
 [13, 1],
 [1, 31],
 [31, 32],
 [32, 33],
 [33, 4],
 [4, 8],
 [8, 34],
 [34, 3],
 [3, 35],
 [35, 36],
 [36, 37],
 [37, 17],
 [17, 38],
 [38, 39],
 [39, 1],
 [1, 40],
 [40, 18],
 [18, 19],
 [19, 6],
 [6, 20],
 [20, 8],
 [8, 41],
 [41, 3],
 [3, 21],
 [21, 15],
 [15, 42],
 [42, 17],
 [17, 43],
 [43, 44],
 [44, 1],
 [1, 3],
 [3, 19],
 [19, 45],
 [45, 46],
 [46, 47],
 [47, 5],
 [5, 48],
 [48, 49],
 [49, 3],
 [3, 5],
 [5, 50],
 [50, 1],
 [1, 51],
 [51, 52],
 [52, 53],
 [53, 54],
 [54, 55],
 [55, 56],
 [56, 22],
 [22, 7],
 [7, 2],
 [2, 57],
 [57, 4],
 [4, 58],
 [58, 1],
 [1, 59],
 [59, 60],
 [60, 2],
 [2, 61],
 [61, 62],
 [62, 20],
 [20, 8],
 [8, 63],
 [63, 64],
 [64, 2],
 [2, 65],
 [65, 66],
 [66, 3],
 [3, 67],
 [67, 4],
 [4, 

In [11]:
# split into X and y elements
sequences=array(sequences)
X,y= sequences[:,0],sequences[:,1]

In [12]:
X[:5],y[:5]

(array([24, 10,  1, 11,  3]), array([10,  1, 11,  3, 12]))

In [13]:
# one hot encode outputs
y= to_categorical(y, num_classes=vocab_size)
y[:5]
            

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [14]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=1))
model.add(LSTM(50))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1, 10)             960       
_________________________________________________________________
lstm (LSTM)                  (None, 50)                12200     
_________________________________________________________________
dense (Dense)                (None, 96)                4896      
Total params: 18,056
Trainable params: 18,056
Non-trainable params: 0
_________________________________________________________________
None


In [15]:
# generate a sequence from the model
def generate_seq(model, tokenizer, seed_text, n_words):
	in_text, result = seed_text, seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		encoded = array(encoded)
		# predict a word in the vocabulary
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text, result = out_word, result + '  ' + out_word
	return result

In [16]:
print(generate_seq(model, tokenizer,"development", 6))

C:\Users\MohamedBabikerMohame\anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


development  words  simple  writes  not  installs  complicated


In [22]:

data = [
    "It is not a lack of love, but a lack of friendship that makes unhappy marriages.",
    "That which does not kill us makes us stronger.",
    "I'm not upset that you lied to me, I'm upset that from now on I can't believe you.",
    "And those who were seen dancing were thought to be insane by those who could not hear the music.",
    "It is hard enough to remember my opinions, without also remembering my reasons for them!"
]


In [26]:
for w in data:
    seq = w[:40].lower()
    print(seq)
    print(generate_seq(model, tokenizer,"and", 4))
   
    print()

it is not a lack of love, but a lack of 
and  words  simple  writes  not

that which does not kill us makes us str
and  words  simple  writes  not

i'm not upset that you lied to me, i'm u
and  words  simple  writes  not

and those who were seen dancing were tho
and  words  simple  writes  not

it is hard enough to remember my opinion
and  words  simple  writes  not

